# **BIG DATA ANALYTICS LAB PROJECT**

## Loan Default Prediciton using classifier models


*   Logistic Regression

*   Decision Tree
*   Random Forest

*   XGBoost




### Liam Bopanna Machianda Muthanna - 210962178
### Pranay Jaju - 210962146

In [ ]:
!pip install pyspark

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,split
from pyspark import SparkConf
conf=SparkConf().set('spark.driver.memory', '16g')
spark=SparkSession.builder.config(conf=conf).getOrCreate()

# Read data

In [ ]:
data= spark.read.option("inferSchema","True").option("header","True").csv("/content/drive/MyDrive/BDA project/accepted_2007_to_2018Q4.csv")


In [ ]:
data.cache()

DataFrame[id: string, member_id: string, loan_amnt: double, funded_amnt: double, funded_amnt_inv: double, term: string, int_rate: double, installment: double, grade: string, sub_grade: string, emp_title: string, emp_length: string, home_ownership: string, annual_inc: string, verification_status: string, issue_d: string, loan_status: string, pymnt_plan: string, url: string, desc: string, purpose: string, title: string, zip_code: string, addr_state: string, dti: string, delinq_2yrs: string, earliest_cr_line: string, fico_range_low: string, fico_range_high: string, inq_last_6mths: string, mths_since_last_delinq: string, mths_since_last_record: string, open_acc: string, pub_rec: string, revol_bal: string, revol_util: string, total_acc: string, initial_list_status: string, out_prncp: string, out_prncp_inv: string, total_pymnt: string, total_pymnt_inv: string, total_rec_prncp: string, total_rec_int: string, total_rec_late_fee: string, recoveries: string, collection_recovery_fee: string, last

In [ ]:
data.printSchema()

root
 |-- id: string (nullable = true)
 |-- member_id: string (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- funded_amnt: double (nullable = true)
 |-- funded_amnt_inv: double (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_title: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- issue_d: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- pymnt_plan: string (nullable = true)
 |-- url: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- title: string (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string 

In [ ]:
selected_features = ['addr_state', 'annual_inc', 'earliest_cr_line', 'emp_length', 'emp_title', 'fico_range_high', 'fico_range_low', 'grade', 'home_ownership', 'application_type',
                    'initial_list_status', 'int_rate', 'loan_amnt', 'num_actv_bc_tl', 'loan_status', 'mort_acc', 'tot_cur_bal', 'open_acc', 'pub_rec', 'pub_rec_bankruptcies',
                    'purpose', 'revol_bal', 'revol_util', 'sub_grade', 'term', 'title', 'total_acc', 'verification_status']

In [ ]:
df = data.select(selected_features)

In [ ]:
df.show()

+----------+----------+----------------+----------+--------------------+---------------+--------------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+---------+----------+--------------------+---------+-------------------+
|addr_state|annual_inc|earliest_cr_line|emp_length|           emp_title|fico_range_high|fico_range_low|grade|home_ownership|application_type|initial_list_status|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|           purpose|revol_bal|revol_util|sub_grade|      term|               title|total_acc|verification_status|
+----------+----------+----------------+----------+--------------------+---------------+--------------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-----

In [ ]:
df.groupBy("loan_status").count().show()

+--------------------+-------+
|         loan_status|  count|
+--------------------+-------+
|          Fully Paid|1076751|
|             Default|     40|
|     In Grace Period|   8436|
|         Charged Off| 268558|
|            Oct-2015|      1|
|  Late (31-120 days)|  21467|
|             Current| 878317|
|   Late (16-30 days)|   4349|
|                NULL|     33|
|Does not meet the...|   1988|
|Does not meet the...|    761|
+--------------------+-------+



Keeping only "Fully Paid" and "Charged Off" values in the loan_status column

In [ ]:
df = df.filter((col('loan_status')=="Fully Paid") | (col('loan_status')== 'Charged Off'))

In [ ]:
df.count()

1345309

In [ ]:
df_len = 1345309

In [ ]:
df.groupBy("loan_status").count().show()

+-----------+-------+
|loan_status|  count|
+-----------+-------+
| Fully Paid|1076751|
|Charged Off| 268558|
+-----------+-------+



In [ ]:
from pyspark.sql.functions import isnull
for i in selected_features:
  # print("COLUMN : ",i)
  # print("Percentage of Null values : ",end = " ")
  no_nulls = df.filter(isnull(df[i])).count()
  if no_nulls:
    print("COLUMN : ",i)
    print("Count of Null values : ",end = " ")
    print(f"{no_nulls:.2f}")
    print("Percentage of Null values : ",end = " ")
    print(f"{no_nulls/df_len*100:.2f}" + "%")
    print("================================================================================")
  #   pass
  # else:
  #   no_nulls = 0
  # print(no_nulls/df_len*100)
  # print("================================================================================")

COLUMN :  addr_state
Count of Null values :  1.00
Percentage of Null values :  0.00%
COLUMN :  earliest_cr_line
Count of Null values :  1.00
Percentage of Null values :  0.00%
COLUMN :  emp_length
Count of Null values :  78511.00
Percentage of Null values :  5.84%
COLUMN :  emp_title
Count of Null values :  85785.00
Percentage of Null values :  6.38%
COLUMN :  fico_range_high
Count of Null values :  1.00
Percentage of Null values :  0.00%
COLUMN :  fico_range_low
Count of Null values :  1.00
Percentage of Null values :  0.00%
COLUMN :  application_type
Count of Null values :  51.00
Percentage of Null values :  0.00%
COLUMN :  initial_list_status
Count of Null values :  14.00
Percentage of Null values :  0.00%
COLUMN :  num_actv_bc_tl
Count of Null values :  67518.00
Percentage of Null values :  5.02%
COLUMN :  mort_acc
Count of Null values :  47260.00
Percentage of Null values :  3.51%
COLUMN :  tot_cur_bal
Count of Null values :  67433.00
Percentage of Null values :  5.01%
COLUMN :  o

In [ ]:
df = df.na.drop()

In [ ]:
df.count()

1180534

In [ ]:
df_len = 1180534

In [ ]:
from pyspark.ml.feature import StringIndexer,OneHotEncoder
def one_hot_encode(df,col_name):
  indexer = StringIndexer(inputCol=col_name, outputCol=col_name+"_Indexed")
  indexerModel = indexer.fit(df)
  indexed_df = indexerModel.transform(df)
  # indexed_df.show()
  encoder = OneHotEncoder(inputCol=col_name+"_Indexed", outputCol=col_name+"_onehot")
  encoded_df = encoder.fit(indexed_df).transform(indexed_df)
  # encoded_df.show(truncate=False)
  encoded_df = encoded_df.drop(col_name+"_Indexed")
  encoded_df = encoded_df.drop(col_name)
  df = encoded_df
  df.show()
  return df

In [ ]:
def cat_var_analysis(df,column_name):
  df.groupBy(column_name,"loan_status").count().orderBy([column_name,"loan_status"]).show()


In [ ]:
def num_var_analysis(df,column_name):
  df.select(column_name).summary( "25%", "75%").show()
  df.select("loan_status",column_name).groupBy("loan_status").agg({column_name:"mean"}).show()
  df.select("loan_status",column_name).groupBy("loan_status").agg({column_name:"count"}).show()

# Data Cleaning and Preprocessing

addr_state


In [ ]:
df = one_hot_encode(df,"addr_state")

+----------+----------------+----------+--------------------+---------------+--------------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+---------+----------+--------------------+---------+-------------------+-----------------+
|annual_inc|earliest_cr_line|emp_length|           emp_title|fico_range_high|fico_range_low|grade|home_ownership|application_type|initial_list_status|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|           purpose|revol_bal|revol_util|sub_grade|      term|               title|total_acc|verification_status|addr_state_onehot|
+----------+----------------+----------+--------------------+---------------+--------------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+--

annual_inc

In [ ]:
num_var_analysis(df,"annual_inc")



+-------+----------+
|summary|annual_inc|
+-------+----------+
|    25%|   48000.0|
|    75%|   93000.0|
+-------+----------+

+-----------+-----------------+
|loan_status|  avg(annual_inc)|
+-----------+-----------------+
| Fully Paid|79690.91517226721|
|Charged Off|72794.54987222192|
+-----------+-----------------+

+-----------+-----------------+
|loan_status|count(annual_inc)|
+-----------+-----------------+
| Fully Paid|           948178|
|Charged Off|           232356|
+-----------+-----------------+



In [ ]:
outlier_range = 90000+1.5*(90000-48000)
df = df.filter(col("annual_inc")<outlier_range)

earliest_cr_line

In [ ]:
from pyspark.sql.types import IntegerType
df = df.withColumn('earliest_cr_line_year', split(df["earliest_cr_line"], '-').getItem(1).cast(IntegerType()))
df = df.drop("earliest_cr_line")


In [ ]:
df.show()

+----------+----------+--------------------+---------------+--------------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+---------+----------+--------------------+---------+-------------------+-----------------+---------------------+
|annual_inc|emp_length|           emp_title|fico_range_high|fico_range_low|grade|home_ownership|application_type|initial_list_status|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|           purpose|revol_bal|revol_util|sub_grade|      term|               title|total_acc|verification_status|addr_state_onehot|earliest_cr_line_year|
+----------+----------+--------------------+---------------+--------------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+-

emp_length


In [ ]:
num_var_analysis(df,"emp_length")

+-------+----------+
|summary|emp_length|
+-------+----------+
|    25%|      NULL|
|    75%|      NULL|
+-------+----------+

+-----------+---------------+
|loan_status|avg(emp_length)|
+-----------+---------------+
| Fully Paid|           NULL|
|Charged Off|           NULL|
+-----------+---------------+

+-----------+-----------------+
|loan_status|count(emp_length)|
+-----------+-----------------+
| Fully Paid|           892415|
|Charged Off|           222856|
+-----------+-----------------+



In [ ]:
df = df.drop("emp_length")

emp_title

In [ ]:
df.select("emp_title").distinct().count()

323194

In [ ]:
df = df.drop("emp_title")

fico

In [ ]:
df = df.withColumn('fico',df["fico_range_high"]+df["fico_range_low"])
df = df.drop("fico_range_high")
df = df.drop("fico_range_low")
df.show()

+----------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+---------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+
|annual_inc|grade|home_ownership|application_type|initial_list_status|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|           purpose|revol_bal|revol_util|sub_grade|      term|               title|total_acc|verification_status|addr_state_onehot|earliest_cr_line_year|  fico|
+----------+-----+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+---------+----------+--------------------+---------+-------------------+-----------------+---------------------+---

Grade and Sub Grade

In [ ]:
df.select("grade").distinct().show()

+-----+
|grade|
+-----+
|    F|
|    E|
|    B|
|    D|
|    C|
|    A|
|    G|
+-----+



In [ ]:
df.select("sub_grade").distinct().show()

+---------+
|sub_grade|
+---------+
|       D5|
|       F2|
|       B4|
|       A2|
|       E4|
|       B2|
|       C3|
|       D1|
|       C4|
|       F1|
|       D3|
|       F5|
|       G2|
|       B1|
|       B3|
|       E5|
|       C5|
|       G3|
|       A4|
|       F4|
+---------+
only showing top 20 rows



In [ ]:
df = df.drop("grade")

In [ ]:
df = one_hot_encode(df,"sub_grade")


+----------+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+----------------+
|annual_inc|home_ownership|application_type|initial_list_status|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|           purpose|revol_bal|revol_util|      term|               title|total_acc|verification_status|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|
+----------+--------------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+----------

home_ownership

In [ ]:
cat_var_analysis(df,"home_ownership")

+--------------+-----------+------+
|home_ownership|loan_status| count|
+--------------+-----------+------+
|           ANY|Charged Off|    45|
|           ANY| Fully Paid|   182|
|      MORTGAGE|Charged Off| 94360|
|      MORTGAGE| Fully Paid|451424|
|          NONE|Charged Off|     6|
|          NONE| Fully Paid|    37|
|         OTHER|Charged Off|     8|
|         OTHER| Fully Paid|    33|
|           OWN|Charged Off| 23284|
|           OWN| Fully Paid| 92225|
|          RENT|Charged Off|105153|
|          RENT| Fully Paid|348514|
+--------------+-----------+------+



In [ ]:
df = one_hot_encode(df,"home_ownership")

+----------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+----------------+---------------------+
|annual_inc|application_type|initial_list_status|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|           purpose|revol_bal|revol_util|      term|               title|total_acc|verification_status|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|
+----------+----------------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+-----------

application type

In [ ]:
# df.groupBy("application_type","loan_status").count().orderBy(["application_type","loan_status"]).show()
df.select("application_type").distinct().show()

+----------------+
|application_type|
+----------------+
|       Joint App|
|      Individual|
+----------------+



In [ ]:
df = df.filter((col("application_type")=="Joint App") | (col("application_type")== 'Individual'))

In [ ]:
cat_var_analysis(df,"application_type")

+----------------+-----------+------+
|application_type|loan_status| count|
+----------------+-----------+------+
|      Individual|Charged Off|217928|
|      Individual| Fully Paid|876458|
|       Joint App|Charged Off|  4928|
|       Joint App| Fully Paid| 15957|
+----------------+-----------+------+



In [ ]:
df = one_hot_encode(df,"application_type")

+----------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+----------------+---------------------+-----------------------+
|annual_inc|initial_list_status|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|           purpose|revol_bal|revol_util|      term|               title|total_acc|verification_status|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|
+----------+-------------------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+--------------

initial_list_status

In [ ]:
cat_var_analysis(df,"initial_list_status")

+-------------------+-----------+------+
|initial_list_status|loan_status| count|
+-------------------+-----------+------+
|                  f|Charged Off| 86924|
|                  f| Fully Paid|348582|
|                  w|Charged Off|135932|
|                  w| Fully Paid|543833|
+-------------------+-----------+------+



In [ ]:
df = df.drop("initial_list_status")

int_rate

In [ ]:
num_var_analysis(df,"int_rate")

+-------+--------+
|summary|int_rate|
+-------+--------+
|    25%|     9.8|
|    75%|   16.02|
+-------+--------+

+-----------+------------------+
|loan_status|     avg(int_rate)|
+-----------+------------------+
| Fully Paid|12.712413204616915|
|Charged Off|15.844952794628382|
+-----------+------------------+

+-----------+---------------+
|loan_status|count(int_rate)|
+-----------+---------------+
| Fully Paid|         892415|
|Charged Off|         222856|
+-----------+---------------+



loan_amnt

In [ ]:
num_var_analysis(df,"loan_amnt")

+-------+---------+
|summary|loan_amnt|
+-------+---------+
|    25%|   8000.0|
|    75%|  20000.0|
+-------+---------+

+-----------+------------------+
|loan_status|    avg(loan_amnt)|
+-----------+------------------+
| Fully Paid|13866.884549228778|
|Charged Off|15575.095465233155|
+-----------+------------------+

+-----------+----------------+
|loan_status|count(loan_amnt)|
+-----------+----------------+
| Fully Paid|          892415|
|Charged Off|          222856|
+-----------+----------------+



In [ ]:
outlier_range = 20000.0+1.5*(20000.0-8000)
df = df.filter(col("loan_amnt")<outlier_range)

num_actv_bc_tl

In [ ]:
num_var_analysis(df,"num_actv_bc_tl")

+-------+--------------+
|summary|num_actv_bc_tl|
+-------+--------------+
|    25%|           2.0|
|    75%|           5.0|
+-------+--------------+

+-----------+-------------------+
|loan_status|avg(num_actv_bc_tl)|
+-----------+-------------------+
| Fully Paid| 3.5601719229947006|
|Charged Off| 3.8175927929877718|
+-----------+-------------------+

+-----------+---------------------+
|loan_status|count(num_actv_bc_tl)|
+-----------+---------------------+
| Fully Paid|               888770|
|Charged Off|               221784|
+-----------+---------------------+



In [ ]:
outlier_range = 5+1.5*(5-2)
df = df.filter(col("num_actv_bc_tl")<outlier_range)

mort_acc

In [ ]:
num_var_analysis(df,"mort_acc")

+-------+--------+
|summary|mort_acc|
+-------+--------+
|    25%|     0.0|
|    75%|     3.0|
+-------+--------+

+-----------+------------------+
|loan_status|     avg(mort_acc)|
+-----------+------------------+
| Fully Paid|1.6434758206767004|
|Charged Off|1.3057824089283319|
+-----------+------------------+

+-----------+---------------+
|loan_status|count(mort_acc)|
+-----------+---------------+
| Fully Paid|         873060|
|Charged Off|         216121|
+-----------+---------------+



In [ ]:
outlier_range = 3+2*(3-0)
df = df.filter(col("mort_acc")<outlier_range)

tot_cur_bal

In [ ]:
num_var_analysis(df,"tot_cur_bal")

+-------+-----------+
|summary|tot_cur_bal|
+-------+-----------+
|    25%|    29072.0|
|    75%|   197540.0|
+-------+-----------+

+-----------+------------------+
|loan_status|  avg(tot_cur_bal)|
+-----------+------------------+
| Fully Paid|131879.58764705612|
|Charged Off|111967.76922684003|
+-----------+------------------+

+-----------+------------------+
|loan_status|count(tot_cur_bal)|
+-----------+------------------+
| Fully Paid|            867696|
|Charged Off|            215350|
+-----------+------------------+



In [ ]:
outlier_range = 1000000
df = df.filter(col("tot_cur_bal")<outlier_range)

open_acc

In [ ]:
num_var_analysis(df,"open_acc")

+-------+--------+
|summary|open_acc|
+-------+--------+
|    25%|     8.0|
|    75%|    14.0|
+-------+--------+

+-----------+------------------+
|loan_status|     avg(open_acc)|
+-----------+------------------+
| Fully Paid|11.355555043227666|
|Charged Off|11.761902992699374|
+-----------+------------------+

+-----------+---------------+
|loan_status|count(open_acc)|
+-----------+---------------+
| Fully Paid|         867500|
|Charged Off|         215324|
+-----------+---------------+



 pub_rec

In [ ]:
num_var_analysis(df,"pub_rec")

+-------+-------+
|summary|pub_rec|
+-------+-------+
|    25%|    0.0|
|    75%|    0.0|
+-------+-------+

+-----------+-------------------+
|loan_status|       avg(pub_rec)|
+-----------+-------------------+
| Fully Paid|0.21158731988472623|
|Charged Off|0.24683732421838717|
+-----------+-------------------+

+-----------+--------------+
|loan_status|count(pub_rec)|
+-----------+--------------+
| Fully Paid|        867500|
|Charged Off|        215324|
+-----------+--------------+



purpose

In [ ]:
cat_var_analysis(df,"purpose")

+------------------+-----------+------+
|           purpose|loan_status| count|
+------------------+-----------+------+
|               car|Charged Off|  1599|
|               car| Fully Paid|  9223|
|       credit_card|Charged Off| 39830|
|       credit_card| Fully Paid|199603|
|debt_consolidation|Charged Off|134597|
|debt_consolidation| Fully Paid|505192|
|       educational| Fully Paid|     1|
|  home_improvement|Charged Off| 11687|
|  home_improvement| Fully Paid| 53771|
|             house|Charged Off|  1239|
|             house| Fully Paid|  4167|
|    major_purchase|Charged Off|  4286|
|    major_purchase| Fully Paid| 17995|
|           medical|Charged Off|  2583|
|           medical| Fully Paid|  9551|
|            moving|Charged Off|  1785|
|            moving| Fully Paid|  5731|
|             other|Charged Off| 12987|
|             other| Fully Paid| 48351|
|  renewable_energy|Charged Off|   164|
+------------------+-----------+------+
only showing top 20 rows



In [ ]:
df = one_hot_encode(df,"purpose")

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|      term|               title|total_acc|verification_status|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+----------+--------------------+---------+-------------------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+
|   55000.0|   13.99| 

title

In [ ]:
df.select("title").distinct().show()

+--------------------+
|               title|
+--------------------+
|      Major purchase|
|             odymeds|
|               Other|
|Credit card refin...|
|         Home buying|
|    Home improvement|
|            Business|
|          Green loan|
|            Vacation|
|    Medical expenses|
|       Car financing|
|  Debt consolidation|
|                SAVE|
|Moving and reloca...|
|Learning and trai...|
|Paying off higher...|
|Simple Loan Until...|
|Trying to come ba...|
|Prescription Drug...|
|  Pay off Lowes Card|
+--------------------+
only showing top 20 rows



In [ ]:
df = df.drop("title")

term

In [ ]:
cat_var_analysis(df,"term")

+----------+-----------+------+
|      term|loan_status| count|
+----------+-----------+------+
| 36 months|Charged Off|128657|
| 36 months| Fully Paid|689963|
| 60 months|Charged Off| 86667|
| 60 months| Fully Paid|177537|
+----------+-----------+------+



In [ ]:
df = one_hot_encode(df,"term")

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-------------------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|verification_status|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-------------------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+
|   55000.0|   13.99|   3600.0|           2.0| Fully Paid|     1.0|   144904

revol_bal

In [ ]:
num_var_analysis(df,"revol_bal")

+-------+---------+
|summary|revol_bal|
+-------+---------+
|    25%|   5867.0|
|    75%|  18479.0|
+-------+---------+

+-----------+------------------+
|loan_status|    avg(revol_bal)|
+-----------+------------------+
| Fully Paid|14687.719078962537|
|Charged Off|14161.315612751017|
+-----------+------------------+

+-----------+----------------+
|loan_status|count(revol_bal)|
+-----------+----------------+
| Fully Paid|          867500|
|Charged Off|          215324|
+-----------+----------------+



revol_util

In [ ]:
num_var_analysis(df,"revol_util")

+-------+----------+
|summary|revol_util|
+-------+----------+
|    25%|      33.8|
|    75%|      70.5|
+-------+----------+

+-----------+-----------------+
|loan_status|  avg(revol_util)|
+-----------+-----------------+
| Fully Paid|51.19093648415006|
|Charged Off|54.79157037766334|
+-----------+-----------------+

+-----------+-----------------+
|loan_status|count(revol_util)|
+-----------+-----------------+
| Fully Paid|           867500|
|Charged Off|           215324|
+-----------+-----------------+



In [ ]:
outlier_range = 70.5+2*(70.5-33.8)
df = df.filter(col("revol_util")<outlier_range)

verification_status

In [ ]:
cat_var_analysis(df,"verification_status")

+-------------------+-----------+------+
|verification_status|loan_status| count|
+-------------------+-----------+------+
|       Not Verified|Charged Off| 49238|
|       Not Verified| Fully Paid|291757|
|    Source Verified|Charged Off| 92729|
|    Source Verified| Fully Paid|343048|
|           Verified|Charged Off| 73350|
|           Verified| Fully Paid|232683|
+-------------------+-----------+------+



In [ ]:
df = one_hot_encode(df,"verification_status")

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+
|   55000.0|   13.99|   3600.0|           2.0| Fully Pa

In [ ]:
df.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+
|   55000.0|   13.99|   3600.0|           2.0| Fully Pa

In [ ]:
df.cache()

DataFrame[annual_inc: string, int_rate: double, loan_amnt: double, num_actv_bc_tl: double, loan_status: string, mort_acc: double, tot_cur_bal: string, open_acc: string, pub_rec: string, pub_rec_bankruptcies: double, revol_bal: string, revol_util: string, total_acc: string, addr_state_onehot: vector, earliest_cr_line_year: int, fico: double, sub_grade_onehot: vector, home_ownership_onehot: vector, application_type_onehot: vector, purpose_onehot: vector, term_onehot: vector, verification_status_onehot: vector]

In [ ]:
from pyspark.sql.types import IntegerType,DoubleType
df1 = df.withColumn("annual_inc", df["annual_inc"].cast(DoubleType()))
df1 = df1.withColumn("tot_cur_bal", df1["tot_cur_bal"].cast(DoubleType()))
df1 = df1.withColumn("open_acc", df1["open_acc"].cast(DoubleType()))
df1 = df1.withColumn("pub_rec", df1["pub_rec"].cast(DoubleType()))
df1 = df1.withColumn("revol_bal", df1["revol_bal"].cast(DoubleType()))
df1 = df1.withColumn("revol_util", df1["revol_util"].cast(DoubleType()))
df1 = df1.withColumn("total_acc", df1["total_acc"].cast(DoubleType()))

In [ ]:
indexer = StringIndexer(inputCol='loan_status', outputCol='loan_status_numeric')
indexer_fitted = indexer.fit(df1)
df1 = indexer_fitted.transform(df1)
df1.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+-------------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|loan_status_numeric|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+----------------

In [ ]:
df1.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+
|   55000.0|   13.99|   3600.0|           2.0| Fully Pa

In [ ]:
df1.select("loan_status_numeric").show()

+-------------------+
|loan_status_numeric|
+-------------------+
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                1.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                0.0|
|                1.0|
|                0.0|
+-------------------+
only showing top 20 rows



# DATA SPLITTING into test and train

In [ ]:
train_data,test_data = df1.randomSplit([0.8,0.2])

# INPUT VECTOR CREATION

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
stages = []
colnames = list(df1.columns)
colnames.remove("loan_status")
colnames.remove("loan_status_numeric")
colnames
input_cols = colnames[:]
vector_assembler = VectorAssembler(inputCols = input_cols,outputCol = "featureVector")
assembled_train_data = vector_assembler.transform(train_data)
assembled_train_data.select("featureVector").show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|featureVector                                                                                                                                                       |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|(120,[0,1,2,3,4,5,6,7,8,9,10,11,24,62,63,83,98,103,112,117],[3800.0,17.86,1125.0,1.0,3.0,3604.0,4.0,1.0,1.0,3604.0,82.0,13.0,1.0,1984.0,1324.0,1.0,1.0,1.0,1.0,1.0])|
|(120,[0,1,2,3,5,6,9,10,11,14,62,63,83,99,103,105,117,119],[6000.0,17.86,2100.0,2.0,135274.0,13.0,1381.0,32.9,13.0,1.0,2012.0,1384.0,1.0,1.0,1.0,1.0,1.0,1.0])       |
|(120,[0,1,2,3,5,6,9,10,11,25,62,63,83,99,103,106,117],[7000.0,17.86,3500.0,1.0,11501.0,8.0,1586.0,29.4,8.0,1.0,2011.0,1324.0,1.0,1.0,1.0,1.0,1.0])                  

In [ ]:
assembled_train_data.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+-------------------+--------------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|loan_status_numeric|       featureVector|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+-

In [ ]:
assembled_train_data=assembled_train_data.na.drop()

In [ ]:
assembled_train_data.filter(assembled_train_data['loan_status_numeric'].isNull()).count()

0

In [ ]:
assembled_train_data.printSchema()

root
 |-- annual_inc: double (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- loan_amnt: double (nullable = true)
 |-- num_actv_bc_tl: double (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- mort_acc: double (nullable = true)
 |-- tot_cur_bal: double (nullable = true)
 |-- open_acc: double (nullable = true)
 |-- pub_rec: double (nullable = true)
 |-- pub_rec_bankruptcies: double (nullable = true)
 |-- revol_bal: double (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: double (nullable = true)
 |-- addr_state_onehot: vector (nullable = true)
 |-- earliest_cr_line_year: integer (nullable = true)
 |-- fico: double (nullable = true)
 |-- sub_grade_onehot: vector (nullable = true)
 |-- home_ownership_onehot: vector (nullable = true)
 |-- application_type_onehot: vector (nullable = true)
 |-- purpose_onehot: vector (nullable = true)
 |-- term_onehot: vector (nullable = true)
 |-- verification_status_onehot: vector (nullable = true)
 |-

In [ ]:
assembled_train_data = assembled_train_data.select(["featureVector","loan_status_numeric"])

In [ ]:
assembled_train_data.show()

+--------------------+-------------------+
|       featureVector|loan_status_numeric|
+--------------------+-------------------+
|(120,[0,1,2,3,4,5...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                1.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                1.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                1.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
|(120,[0,1,2,3,5,6...|                0.0|
+----------

In [ ]:
from pyspark.sql.types import StructType, StructField

# Change column metadata
fields = [StructField(field.name,
          field.dataType,
          False) for field in assembled_train_data.schema.fields]

# Store changed data frame in new_schema
new_schema = StructType(fields)
assembled_train_data = spark.createDataFrame(assembled_train_data.rdd,
                           new_schema)
assembled_train_data.printSchema()

root
 |-- featureVector: vector (nullable = false)
 |-- loan_status_numeric: double (nullable = false)



Creating function to get model metrics

In [ ]:
def printAllMetrics(predictionsDF):
    TP = predictionsDF.select("loan_status_numeric", "prediction").filter("loan_status_numeric == 1.0 and prediction == 1.0").count()
    FN = predictionsDF.select("loan_status_numeric", "prediction").filter("loan_status_numeric == 1.0 and prediction == 0.0").count()
    TN = predictionsDF.select("loan_status_numeric", "prediction").filter("loan_status_numeric == 0.0 and prediction == 0.0").count()
    FP = predictionsDF.select("loan_status_numeric", "prediction").filter("loan_status_numeric == 0.0 and prediction == 1.0").count()

    precision_p   = TP/(TP+FP)
    precision_n   = TN/(TN+FN)
    recall_p      = TP/(TP+FN)
    recall_n      = TN/(TN+FP)
    f1_p          = 2*precision_p*recall_p/(precision_p+recall_p)
    f1_n          = 2*precision_n*recall_n/(precision_n+recall_n)
    avg_precision = (precision_p*(TP+FN)+precision_n*(TN+FP))/(TP+FN+TN+FP)
    avg_recall    = (recall_p*(TP+FN)+recall_n*(TN+FP))/(TP+FN+TN+FP)
    avg_f1        = (f1_p*(TP+FN)+f1_n*(TN+FP))/(TP+FN+TN+FP)
    accuracy      = (TN+TP)/(TP+FN+TN+FP)

    # print('Precision of True    ', precision_p)
    # print('Precision of False   ', precision_n)
    print('AVERAGE PRECISION     ', f"{avg_precision*1:.2f}")
    # print('Recall of True       ', recall_p)
    # print('Recall of False      ', recall_n)
    print('AVERAGE RECALL        ', f"{avg_recall*1:.2f}")
    # print('F1 of True           ', f1_p)
    # print('F1 of False          ', f1_n)
    print('AVERAGE F1             ', f"{avg_f1*1:.2f}")
    print('ACCURACY               ', f"{accuracy*100:.2f}%")

# LOGISTIC REGRESSION

In [ ]:
from pyspark.ml.classification import LogisticRegression
assembled_train_data.persist()
log_reg = LogisticRegression(featuresCol='featureVector', labelCol='loan_status_numeric')
model = log_reg.fit(assembled_train_data)


In [ ]:
assembled_test_data = vector_assembler.transform(test_data)
assembled_train_data = assembled_train_data.select(["featureVector","loan_status_numeric"])
assembled_test_data.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+-------------------+--------------------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|loan_status_numeric|       featureVector|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+-

In [ ]:
results = model.transform(assembled_test_data )
results.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+-------------------+--------------------+--------------------+--------------------+----------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|loan_status_numeric|       featureVector|       rawPrediction|         probability|prediction|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+----------------

Model Metrics

In [ ]:
printAllMetrics(results)

AVERAGE PRECISION      0.76
AVERAGE RECALL         0.80
AVERAGE F1              0.73
ACCURACY                80.27%


# DECISION TREE

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

classifier = DecisionTreeClassifier(seed=1234,labelCol="loan_status_numeric",featuresCol="featureVector",predictionCol="prediction")
model=classifier.fit(assembled_train_data)
print(model.toDebugString)

DecisionTreeClassificationModel: uid=DecisionTreeClassifier_7c1ca8242668, depth=5, numNodes=11, numClasses=2, numFeatures=120
  If (feature 1 <= 13.465)
   Predict: 0.0
  Else (feature 1 > 13.465)
   If (feature 117 <= 0.5)
    If (feature 1 <= 17.369999999999997)
     Predict: 0.0
    Else (feature 1 > 17.369999999999997)
     If (feature 99 <= 0.5)
      Predict: 0.0
     Else (feature 99 > 0.5)
      If (feature 1 <= 24.034999999999997)
       Predict: 0.0
      Else (feature 1 > 24.034999999999997)
       Predict: 1.0
   Else (feature 117 > 0.5)
    Predict: 0.0



In [ ]:
results = model.transform(assembled_test_data )
results.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+-------------------+--------------------+------------------+--------------------+----------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|loan_status_numeric|       featureVector|     rawPrediction|         probability|prediction|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+--------------------

Model Metrics

In [ ]:
printAllMetrics(results)

AVERAGE PRECISION      0.75
AVERAGE RECALL         0.80
AVERAGE F1              0.72
ACCURACY                80.13%


# RANDOM FOREST CLASSIFIER

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
model = RandomForestClassifier(labelCol="loan_status_numeric",featuresCol="featureVector",predictionCol="prediction")
model=classifier.fit(assembled_train_data)

In [ ]:
results = model.transform(assembled_test_data)
results.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+-------------------+--------------------+------------------+--------------------+----------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|loan_status_numeric|       featureVector|     rawPrediction|         probability|prediction|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+--------------------

Model Metrics

In [ ]:
printAllMetrics(results)

AVERAGE PRECISION      0.75
AVERAGE RECALL         0.80
AVERAGE F1              0.72
ACCURACY                80.13%


# GRADIENT BOOSTING CLASSIFIER

In [ ]:
from pyspark.ml.classification import GBTClassifier
model = BTClassifier(labelCol="loan_status_numeric",featuresCol="featureVector",predictionCol="prediction")
model=classifier.fit(assembled_train_data)

In [ ]:
results = model.transform(assembled_test_data )
results.show()

+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+---------------------+------+----------------+---------------------+-----------------------+--------------+-------------+--------------------------+-------------------+--------------------+------------------+--------------------+----------+
|annual_inc|int_rate|loan_amnt|num_actv_bc_tl|loan_status|mort_acc|tot_cur_bal|open_acc|pub_rec|pub_rec_bankruptcies|revol_bal|revol_util|total_acc|addr_state_onehot|earliest_cr_line_year|  fico|sub_grade_onehot|home_ownership_onehot|application_type_onehot|purpose_onehot|  term_onehot|verification_status_onehot|loan_status_numeric|       featureVector|     rawPrediction|         probability|prediction|
+----------+--------+---------+--------------+-----------+--------+-----------+--------+-------+--------------------+---------+----------+---------+-----------------+--------------------

Model Metrics

In [ ]:
printAllMetrics(results)

AVERAGE PRECISION      0.75
AVERAGE RECALL         0.80
AVERAGE F1              0.72
ACCURACY                80.13%


# CONCLUSION

All models have good accuracies around the 80% range and provide a safe estimation of the risk of each loan